# **IP4_Run2**

In [9]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk, which is required to install pyproject.toml-based projects


In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business."""

In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

Variables:
1. Full-time employee hours (Expected: 800 hours)
2. Full-time working hours deviation (positive and negative)
3. Part-time employee hours (Expected: 320 hours)
4. Part-time working hours deviation (positive and negative)
5. Total number of units sold (Goal: 5500 units)
6. Sales deviation (positive and negative)
7. Full-time employee overtime hours (Target: 100 hours)
8. Full-time employee overtime deviation (positive and negative)
9. Part-time employee overtime hours 
10. General overtime hours deviation
11. Priority factor for each deviation (P1 [Sales], P2 [FTE Overtime], P3 [FTE Hours], P4 [General Overtime])
12. Relative concern factor for total full-time employee hours 
13. Relative cost factor for overtime hours of part-time employees.


### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response2.choices[0].message.content)

Minimize Z = P1*(Sales Target - Units Sold)^2 + P2*(Overtime Target FT - Overtime Hours FT)^2 + P3*(Full Time Hours Target - Full Time Hours)^2 + P3*(Part Time Hours Target - Part Time Hours)^2 + 2*P3*(Full Time Hours Target - Full Time Hours)^2 + P4*(Total Overtime Target - Total Overtime Hours)^2 + 3*P4*(Part Time Overtime Hours)^2


In [11]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
print(response3.choices[0].message.content)

1. 0 <= Full-time employee hours <= 800 + Positive full-time working hours deviation - Negative full-time working hours deviation

2. 0 <= Part-time employee hours <= 320 + Positive part-time working hours deviation - Negative part-time working hours deviation

3. Units Sold = 5 * Full-time employee hours + 2 * Part-time employee hours + Positive sales deviation - Negative sales deviation

4. Units Sold >= 5500

5. 0 <= Full-time employee overtime hours <= 100 + Positive full-time overtime hours deviation - Negative full-time overtime hours deviation

6. Part-time employee overtime hours >= 0

7. Total Overtime Hours = Full-time employee overtime hours + Part-time employee overtime hours + General overtime hours deviation

8. P1, P2, P3, P4 >= 0

9. Negative full-time working hours deviation, Negative part-time working hours deviation, Negative sales deviation, Negative full-time overtime hours deviation >=0

10. Positive full-time working hours deviation, Positive part-time working ho

In [13]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [14]:
Markdown(response4.choices[0].message.content)

Here's a sample implementation using Pyomo in Python:

```python
from pyomo.environ import *

model = ConcreteModel()

# Sample data and priorities
FT_hrs_target = 800
PT_hrs_target = 320
sales_target = 5500
overtime_target = 100
priorities = dict(P1=1, P2=1, P3=1, P4=1)

# Variables
model.FT_hrs = Var(domain=NonNegativeReals)
model.FT_deviation = Var(domain=Reals)
model.PT_hrs = Var(domain=NonNegativeReals)
model.PT_deviation = Var(domain=Reals)
model.sales = Var(domain=NonNegativeReals)
model.sales_deviation = Var(domain=Reals)
model.overtime = Var(domain=NonNegativeReals)
model.overtime_deviation = Var(domain=Reals)

# Objective: Minimize total deviation
model.objective = Objective(expr=(
    priorities['P1'] * (sales_target - model.sales - model.sales_deviation)**2 + 
    priorities['P2'] * (overtime_target - model.overtime - model.overtime_deviation)**2 + 
    priorities['P3'] * (FT_hrs_target - model.FT_hrs - model.FT_deviation)**2 + 
    priorities['P3'] * (PT_hrs_target - model.PT_hrs - model.PT_deviation)**2 + 
    2 * priorities['P3'] * (FT_hrs_target - model.FT_hrs - model.FT_deviation)**2 + 
    priorities['P4'] * (overtime_target - model.overtime - model.overtime_deviation)**2), 
    sense=minimize)

# Constraints
model.constraints = ConstraintList()

# 1 and 2 - Full-time and Part-time working hours boundaries
model.constraints.add(expr=model.FT_hrs <= FT_hrs_target + model.FT_deviation)
model.constraints.add(expr=model.PT_hrs <= PT_hrs_target + model.PT_deviation)

# 3 and 4 - Total units sold
model.constraints.add(expr=model.sales == 5 * model.FT_hrs + 2 * model.PT_hrs + model.sales_deviation)
model.constraints.add(expr=model.sales >= sales_target)

# 5 and 6 - Overtime hours
model.constraints.add(expr=model.overtime <= overtime_target + model.overtime_deviation)
model.constraints.add(expr=model.overtime >= 0)
                  
# 7 - Deviations cannot be negative
model.constraints.add(expr=model.FT_deviation >= 0)
model.constraints.add(expr=model.PT_deviation >= 0)
model.constraints.add(expr=model.sales_deviation >= 0)
model.constraints.add(expr=model.overtime_deviation >= 0)

# Solve
SolverFactory('glpk').solve(model)

# print results
print('FT hours:', model.FT_hrs.value)
print('PT hours:', model.PT_hrs.value)
print('Sales:', model.sales.value)
print('Overtime:', model.overtime.value)
print('Objective:', model.objective())
```
This code is only a sample and may differ depending on the specific needs of your problem. It assumes the GLPK solver is installed, which is an open-source solver for linear programming problems. It might need to be adapted depending on the specific requirements and constraints of your optimization problem.

### **Run the code Generated by GPT4**

In [16]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.2 MB/s eta 0:00:00


In [17]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [18]:
from pyomo.environ import *

model = ConcreteModel()

# Sample data and priorities
FT_hrs_target = 800
PT_hrs_target = 320
sales_target = 5500
overtime_target = 100
priorities = dict(P1=1, P2=1, P3=1, P4=1)

# Variables
model.FT_hrs = Var(domain=NonNegativeReals)
model.FT_deviation = Var(domain=Reals)
model.PT_hrs = Var(domain=NonNegativeReals)
model.PT_deviation = Var(domain=Reals)
model.sales = Var(domain=NonNegativeReals)
model.sales_deviation = Var(domain=Reals)
model.overtime = Var(domain=NonNegativeReals)
model.overtime_deviation = Var(domain=Reals)

# Objective: Minimize total deviation
model.objective = Objective(expr=(
    priorities['P1'] * (sales_target - model.sales - model.sales_deviation)**2 +
    priorities['P2'] * (overtime_target - model.overtime - model.overtime_deviation)**2 +
    priorities['P3'] * (FT_hrs_target - model.FT_hrs - model.FT_deviation)**2 +
    priorities['P3'] * (PT_hrs_target - model.PT_hrs - model.PT_deviation)**2 +
    2 * priorities['P3'] * (FT_hrs_target - model.FT_hrs - model.FT_deviation)**2 +
    priorities['P4'] * (overtime_target - model.overtime - model.overtime_deviation)**2),
    sense=minimize)

# Constraints
model.constraints = ConstraintList()

# 1 and 2 - Full-time and Part-time working hours boundaries
model.constraints.add(expr=model.FT_hrs <= FT_hrs_target + model.FT_deviation)
model.constraints.add(expr=model.PT_hrs <= PT_hrs_target + model.PT_deviation)

# 3 and 4 - Total units sold
model.constraints.add(expr=model.sales == 5 * model.FT_hrs + 2 * model.PT_hrs + model.sales_deviation)
model.constraints.add(expr=model.sales >= sales_target)

# 5 and 6 - Overtime hours
model.constraints.add(expr=model.overtime <= overtime_target + model.overtime_deviation)
model.constraints.add(expr=model.overtime >= 0)

# 7 - Deviations cannot be negative
model.constraints.add(expr=model.FT_deviation >= 0)
model.constraints.add(expr=model.PT_deviation >= 0)
model.constraints.add(expr=model.sales_deviation >= 0)
model.constraints.add(expr=model.overtime_deviation >= 0)

# Solve
SolverFactory('knitro').solve(model)

# print results
print('FT hours:', model.FT_hrs.value)
print('PT hours:', model.PT_hrs.value)
print('Sales:', model.sales.value)
print('Overtime:', model.overtime.value)
print('Objective:', model.objective())

ERROR:pyomo.opt:Solver (asl) returned non-zero return code (1)
ERROR:pyomo.opt:Solver log:

Sorry, a demo license is limited to 10 variables and
10 constraints and objectives.
You have 8 variables, 10 constraints, and 1 objective.

Contact us at <licensing@ampl.com> or https://discuss.ampl.com/
for more information.
Artelys Knitro 14.0.0: 


ApplicationError: Solver (asl) did not exit normally

In [19]:
from pyomo.environ import *

model = ConcreteModel()

# Sample data and priorities
FT_hrs_target = 800
PT_hrs_target = 320
sales_target = 5500
overtime_target = 100
priorities = dict(P1=4, P2=3, P3=2, P4=1)

# Variables
model.FT_hrs = Var(domain=NonNegativeReals)
model.FT_deviation = Var(domain=Reals)
model.PT_hrs = Var(domain=NonNegativeReals)
model.PT_deviation = Var(domain=Reals)
model.sales = Var(domain=NonNegativeReals)
model.sales_deviation = Var(domain=Reals)
model.overtime = Var(domain=NonNegativeReals)
model.overtime_deviation = Var(domain=Reals)

# Objective: Minimize total deviation
model.objective = Objective(expr=(
    priorities['P1'] * (sales_target - model.sales - model.sales_deviation)**2 +
    priorities['P2'] * (overtime_target - model.overtime - model.overtime_deviation)**2 +
    priorities['P3'] * (FT_hrs_target - model.FT_hrs - model.FT_deviation)**2 +
    priorities['P3'] * (PT_hrs_target - model.PT_hrs - model.PT_deviation)**2 +
    2 * priorities['P3'] * (FT_hrs_target - model.FT_hrs - model.FT_deviation)**2 +
    priorities['P4'] * (overtime_target - model.overtime - model.overtime_deviation)**2),
    sense=minimize)

# Constraints
model.constraints = ConstraintList()

# 1 and 2 - Full-time and Part-time working hours boundaries
model.constraints.add(expr=model.FT_hrs <= FT_hrs_target + model.FT_deviation)
model.constraints.add(expr=model.PT_hrs <= PT_hrs_target + model.PT_deviation)

# 3 and 4 - Total units sold
model.constraints.add(expr=model.sales == 5 * model.FT_hrs + 2 * model.PT_hrs + model.sales_deviation)
model.constraints.add(expr=model.sales >= sales_target)

# 5 and 6 - Overtime hours
model.constraints.add(expr=model.overtime <= overtime_target + model.overtime_deviation)
model.constraints.add(expr=model.overtime >= 0)

# 7 - Deviations cannot be negative
model.constraints.add(expr=model.FT_deviation >= 0)
model.constraints.add(expr=model.PT_deviation >= 0)
model.constraints.add(expr=model.sales_deviation >= 0)
model.constraints.add(expr=model.overtime_deviation >= 0)

# Solve
SolverFactory('knitro').solve(model)

# print results
print('FT hours:', model.FT_hrs.value)
print('PT hours:', model.PT_hrs.value)
print('Sales:', model.sales.value)
print('Overtime:', model.overtime.value)
print('Objective:', model.objective())

ERROR:pyomo.opt:Solver (asl) returned non-zero return code (1)
ERROR:pyomo.opt:Solver log:

Sorry, a demo license is limited to 10 variables and
10 constraints and objectives.
You have 8 variables, 10 constraints, and 1 objective.

Contact us at <licensing@ampl.com> or https://discuss.ampl.com/
for more information.
Artelys Knitro 14.0.0: 


ApplicationError: Solver (asl) did not exit normally

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**